# <font color="red"> MBA em IA e Big Data</font>
## <span style="color:red">Redes Neurais e Deep Learning</span>


## Keras para Implementação de Redes Neurais Profundas

*Roseli Aparecida Francelin Romero*<br>
*ICMC/USP São Carlos*

### Estudo de caso: dados estruturados
### Classificação de Estados segundo sua população, UF e PIB

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import train_test_split
#
# Fixando uma semente geradora
keras.utils.set_random_seed(2) # para garantir reprodutibilidade
#
# carregando base com Pandas
# Este  arquivo esta disponível na página do curso para download
df = pd.read_csv("vw_pib_percapita_clean.csv")
df = df.dropna()

AttributeError: module 'keras.api._v2.keras.utils' has no attribute 'set_random_seed'

In [ ]:
df

Vamos obter uma codificação numérica para o estado (UF)

In [ ]:
ufcode = df['UF'].astype("category").cat.codes

df.insert(1,"UF_code", ufcode.astype(float))
df

### Discriminar cidades 1 e 2 das demais (3+)

In [ ]:
df['classe'].value_counts().plot(kind = 'bar')

In [ ]:
df['classe'].value_counts()

#### Passos preliminares
* converter o dataframe para numpy array 
* separar as features (entrada) dos alvos (saída)

In [ ]:
nparray = df.to_numpy()

In [ ]:
features = (nparray[:,1:-1]).astype(float)
targets = (nparray[:,-1] > 2).astype(int)  ## cria problema binário
print("Features = ", features.shape)
print("Targets = ", targets.shape)

### 1) Preparando conjuntos de treinamento e validação

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(features, targets, 
                                     test_size=0.25, random_state=0)

In [ ]:
num_inst_teste = int(len(features)*0.2)
print("Exemplos de treinamento:", len(X_train))
print("Exemplos de validacao:", len(X_val))

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
counts_train = np.bincount(y_train)
counts_val = np.bincount(y_val)

print("Porcentagem da classe minoritária treinamento: {} ({:.2f}% of total)".format(
        counts_train[1], 100 * float(counts_train[1]) / len(y_train)
    ))

print("Porcentagem da classe minoritária validacao: {} ({:.2f}% of total)".format(
        counts_val[1], 100 * float(counts_val[1]) / len(y_val)
    ))

## 2) Montando a rede neural para classificação

Utilizaremos primeiro uma rede rasa ("MLP" com uma camada)

In [ ]:
model1 = keras.Sequential(
 [
   keras.layers.Dense(8, activation="sigmoid", input_shape=(X_train.shape[-1],)),
   keras.layers.Dense(1, activation="sigmoid"),
 ]
)

model1.summary()

# 3 x 8 = 24 + 8 = 32
# 8 x 1 = 8 + 1 = 9

**Avaliação**: a base é muito desbalanceada portanto:
* Falsos positivos / negativos
* Verdadeiros positivos / negativos
* Precisão e Revocação

**Preparação do modelo**: otimizadores e outros

In [ ]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

# compilamos o modelo utilizando:
#- um otimizador SGD
#- a função de perda entropia cruzada
#- as métricas acima
model1.compile(
    optimizer=keras.optimizers.SGD(), loss="mse", metrics=metrics
)

**Treinamento do modelo**

Aqui aparecem:
* **batchsize**
* número de épocas (**epochs**). 

Uma época ocorre após as iterações (cada batch) completarem<br>
o total de exemplos de treinamento

* Temos 4174 exemplos de treinamento. 
* Se usarmos batchsize = 10, 
* 1 época precisa de 4174/10 = 417 iterações (mini-batchs selecionados)


In [ ]:
batch_size = 10
epochs = 30

In [ ]:
len(X_train)/batch_size

In [ ]:
history1 = model1.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,  ## mostra as iteracoes
    validation_data=(X_val, y_val),
)

Com o Keras podemos plotar as funções de perda ao longo do treinamento

In [ ]:
plt.plot(history1.history["loss"])
plt.plot(history1.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "val"], loc="upper left")
plt.show()

In [ ]:
# Computando as métricas para a validacao
score = model1.evaluate(X_val, y_val, verbose=0)

In [ ]:
print("Perda (mse): ", score[0])
print("Falsos negativos: ", score[1])
print("Falsos positivos: ", score[2])
print("Verdadeiros negativos: ", score[3])
print("Verdadeiros positivos: ", score[4])
print("Precisao: ", score[5])
print("Revocacao: ", score[6])

## 3) Melhorando o classificador

1. Ponderar as classes pois estão muito desbalanceadas
2. Padronizar os atributos


In [ ]:
# padronizacao z-score
# calculamos média e desvio no treinamento e aplicamos em treinamento e teste
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train -= mean
X_val -= mean

X_train /= std
X_val /= std

In [ ]:
# vamos ponderar o total de cada classe e formar um peso
peso_0 = round(1.0 - (counts_train[0]/len(X_train)), 2)
peso_1 = round(1.0 - (counts_train[1]/len(X_train)), 2)
print("pesos: ", peso_0, peso_1)

# ponderacao das classes
class_weight = {0: peso_0, 1: peso_1}

In [ ]:
model2 = keras.Sequential(
   [
     keras.layers.Dense(8, activation="sigmoid", input_shape=(X_train.shape[-1],)),
     keras.layers.Dense(1, activation="sigmoid"),
   ]
)
model2.summary()

# compilamos o modelo utilizando:
#- um otimizador SGD
#- a função de perda entropia cruzada
#- as métricas acima
model2.compile(
    optimizer=keras.optimizers.SGD(), loss="mse", metrics=metrics
)

history2 = model2.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_val, y_val),
    class_weight=class_weight,
)

In [ ]:
# Computando as métricas para a validacao
score2 = model2.evaluate(X_val, y_val, verbose=0)
print("Falsos negativos: ", score2[1])
print("Falsos positivos: ", score2[2])
print("Verdadeiros negativos: ", score2[3])
print("Verdadeiros positivos: ", score2[4])
print("Precisao: ", score2[5])
print("Revocacao: ", score2[6])

## 4) Adicionando mais camadas


In [ ]:
model3 = keras.Sequential(
 [
    keras.layers.Dense(8, activation="sigmoid", input_shape=(X_train.shape[-1],)),
    keras.layers.Dense(10, activation="sigmoid"),
    keras.layers.Dense(4, activation="sigmoid"),
    keras.layers.Dense(1, activation="sigmoid"),
 ]
)
model3.summary()

# compilamos o modelo utilizando:
#- um otimizador SGD
#- a função de perda entropia cruzada
#- as métricas acima
model3.compile(
    optimizer=keras.optimizers.SGD(), loss="mse", metrics=metrics
)

history3 = model3.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_val, y_val),
    class_weight=class_weight,
)

In [ ]:
plt.plot(history2.history["loss"], '-b', label="1L-train")
plt.plot(history3.history["loss"], '-g', label="2L-train")
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Computando as métricas para o teste
score3 = model3.evaluate(X_val, y_val, verbose=0)

print("Rede rasa")
print("--------")
print("Falsos negativos: ", score2[1])
print("Falsos positivos: ", score2[2])
print("Verdadeiros negativos: ", score2[3])
print("Verdadeiros positivos: ", score2[4])
print("Precisao: %.3f" % score2[5])
print("Revocacao: %.3f\n" % score2[6])

print("Rede profunda")
print("--------")
print("Falsos negativos: ", score3[1])
print("Falsos positivos: ", score3[2])
print("Verdadeiros negativos: ", score3[3])
print("Verdadeiros positivos: ", score3[4])
print("Precisao: %.3f" % score3[5])
print("Revocacao: %.3f" % score3[6])